In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import cftime
import datetime
import climextremes
#import bottleneck

# GEV

In [139]:
# copied from  /global/homes/m/mwehner/pyfiles/make_rv_climextremes.py
# Adapted for numpy/ma/cdms2 by convertcdms.py
# Calculate return value from many slices of a long control run. 
# Use these to calculate a rv

# arguments
# python make_rv_climextremes.py 
# 1. varname 
# 2. sftlf_filename 
# 3. return_period 
# 4. covariate 
# 5. covariate_file 
# 6. multiplier 
# 7. year1 
# 8. year2 
# 9. root_outfile_name 
# 10. filenames
import cdms2, sys, MV2, locale, numpy, climextremes, cdutil, cdtime, string
print ("python "+ " ".join(sys.argv))
#filenames=sys.argv[10:]
#root_outfile_name=sys.argv[9]
#varname=sys.argv[1]
#sftlf_file=cdms2.open(sys.argv[2])
#sftlf=sftlf_file('sftlf')

#year1=sys.argv[7]
#year2=sys.argv[8]


#RP=sys.argv[3]
#T=locale.atof(RP)
# multiplier
#multiplier=locale.atof(sys.argv[6])

#n_realizations=len(filenames)
f=cdms2.open("test_data/lowres_ts_ann_1980-1999.nc")
tim=f['time']

u=f.getdimensionunits('time')
x=f(varname)
tim=x.getAxis(0)
latitude=f.dimensionarray('lat')
longitude=f.dimensionarray('lon')
latitude=latitude.astype(float)
longitude=longitude.astype(float)
nt=len(tim)
nlat=latitude.shape[0]
nlon=longitude.shape[0]
f.close()

r = cdtime.reltime(0,tim.units)
xxyear1=int(1.+r.torel('years since 1').value+tim[0])
xxyear2=int(1.+r.torel('years since 1').value+tim[nt-1])
xyear1=str(xxyear1).zfill(4)
xyear2=str(xxyear2).zfill(4)

#covariate=covariate.co2.to_numpy()
sftlf=sftlf.sftlf.to_numpy()
#covariate_name=sys.argv[4]
#covariate_file=cdms2.open(sys.argv[5])
#covariate=covariate_file(covariate_name, time=(xyear1+'-01-01',xyear2+'-12-31'))
#if covariate_name=='co2mass': covariate=cdutil.YEAR(covariate)
#l_cov=len(covariate)

#if nt!=len(covariate):
#    print ("Error: covariate length",len(covariate)," not equal to variable time dimension length",nt)
#    exit()

# Allocate temporary arrays
nsample=nt*n_realizations
var=MV2.zeros((nsample,nlat,nlon))
replicateIndex=MV2.ones(nsample)
covariates=MV2.zeros(nsample)

# Allocate output arrays
return_value=1.e20*MV2.ones((nt,nlat,nlon))
standard_error=1.e20*MV2.ones((nt,nlat,nlon))

# Calculate the return value o
# Concatenate the ensemble runs for each time slice period
n=0
sample=0
while n<n_realizations:
    f=cdms2.open("test_data/lowres_ts_ann_1980-1999.nc")
    x=f(varname)
    if nt!=len(x):
     print ("Error: variable length error in ".filenames[n])
     exit()
    var[sample:sample+nt,:,:]=x
    replicateIndex[sample:sample+nt]=n+replicateIndex[sample:sample+nt]
    covariates[sample:sample+nt]=covariate
    f.close()
    sample=sample+nt
    n=n+1
var=multiplier*numpy.asarray(var)

# Don't waste time on Antarctica
i=0
istart=0
while i<nlat:
 istart=i
 if latitude[i]>-60.: i=nlat
 i=i+1

# Calculate the twenty year return value of each time slice period
i=istart
while i<var.shape[1]:
 j=0
 while j<var.shape[2]:
  if sftlf[i,j]>50.:
    if MV2.sum(var[:,i,j])==0.0: return_value[:,i,j]=0.0
    else:
     tmp = climextremes.fit_gev(var[:,i,j],covariates, returnPeriod=T, nReplicates = n_realizations, replicateIndex = replicateIndex, locationFun = 1, xNew=covariate)
     if tmp['info']['failure'][0]==0:
      return_value[:,i,j] = tmp['returnValue']/multiplier
      # older results may need to be divided by multiplier (it was an omission bug)
      standard_error[:,i,j] = tmp['se_returnValue']/multiplier
  j=j+1
 i=i+1

#Output return values
return_value.id=varname+'_return_value'
return_value.units=x.units
return_value.setAxisList(x.getAxisList())

standard_error.id=varname+'rv_standard_error'
standard_error.units=x.units
standard_error.setAxisList(x.getAxisList())

#fout=cdms2.open(varname+'_'+RP+'_return_value_climextremes_'+root_outfile_name+'.nc','w')
#fout.execute_line="python "+ " ".join(sys.argv)
#fout.write(return_value)
#fout.write(standard_error)
#fout.return_period=RP
#fout.sample_size=nsample
#fout.nReplicates=n_realizations
#fout.covariate=covariate_name
#covariate.id=covariate_name
#covariate.setAxis(0,x.getAxis(0))
#fout.write(covariate)
#fout.close()

python /home/ordonez4/miniconda3/envs/pcmdi_metrics_dev/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/ordonez4/.local/share/jupyter/runtime/kernel-1a9031ac-0cfe-41d1-9888-9acf70c4759b.json


In [140]:
return_value

TS_return_value
masked_array(
  data=[[[82.64345778, 85.02868971, 81.32701757, 82.87709007],
         [83.05770263, 83.3445483 , 82.57903684, 81.68965969],
         [81.62610326, 81.00620058, 88.70181199, 82.61865317],
         [89.04274638, 83.32677505, 83.35590228, 80.02087528],
         [87.96314232, 82.21598684, 82.51506403, 83.11250725]],

        [[82.85074896, 85.18976052, 81.73324337, 83.05401293],
         [83.25098303, 83.52526984, 82.76886624, 81.9199468 ],
         [81.75966906, 81.32044697, 88.88589249, 82.72286093],
         [89.42093979, 83.38393541, 83.56285765, 80.45680495],
         [88.14499763, 82.37830892, 82.66208258, 83.32221132]],

        [[83.05804015, 85.35083134, 82.13946917, 83.23093578],
         [83.44426342, 83.70599137, 82.95869563, 82.15023391],
         [81.89323486, 81.63469335, 89.06997298, 82.82706868],
         [89.7991332 , 83.44109578, 83.76981302, 80.89273462],
         [88.32685294, 82.54063101, 82.80910112, 83.53191538]],

        [[83.265331

In [141]:
standard_error

TSrv_standard_error
masked_array(
  data=[[[ 3.28526036,  7.64236982,  3.01107129,  3.48279475],
         [ 4.19407318,  3.3417499 ,  3.95301114,  3.52111501],
         [ 5.21579252,  3.15497036, 13.42898237,  3.9819312 ],
         [15.8883879 ,  3.75231515,  5.03021995,  2.6484432 ],
         [15.07074649,  3.8498398 ,  4.17831396,  3.6689266 ]],

        [[ 3.26567417,  7.64885838,  2.91671032,  3.3860691 ],
         [ 4.13797369,  3.26410708,  3.9097003 ,  3.4300535 ],
         [ 5.20253226,  3.13418836, 13.43874949,  3.9477521 ],
         [15.89775995,  3.7392551 ,  5.01295566,  2.62728105],
         [14.98212049,  3.7957644 ,  4.14038751,  3.60103693]],

        [[ 3.25196469,  7.65626651,  2.83027988,  3.29746248],
         [ 4.08645056,  3.19421581,  3.87398368,  3.34535499],
         [ 5.19064477,  3.11663788, 13.4489176 ,  3.91728096],
         [15.90761234,  3.7307326 ,  5.00024136,  2.61015177],
         [14.89474438,  3.74722259,  4.10664014,  3.53999397]],

        [[ 3.24

In [4]:
# Fake data
times = xr.cftime_range(start="1980-01-01", end="1999-12-31", freq="Y", calendar="proleptic_gregorian", name="time")

fake_co2 = np.arange(335,366,(365-335)/19)
fake_co2 = xr.Dataset({'co2': xr.DataArray(
                    data = fake_co2,
                    dims = ["time"],
                    coords = {"time": times},
                    attrs = {"_FillValue": -999,
                            "units" : ""})})
fake_co2.to_netcdf("test_data/fake_cov.nc")

values = np.ones((len(times),5,4)) * (np.arange(1,len(times)+1).reshape(len(times),1,1))/len(times) * 4
values = values + np.random.randint(66,79,size=(len(times),5,4))
values[8,:,:] = 90 # Seem to need an extreme value for gev to converge
fake_lowres = xr.Dataset({'TS': xr.DataArray(
                    data   = values,   # enter data here
                    dims   = ['time',"lat","lon"],
                    coords = {'time': times, "lat": np.arange(0.5,5.5), "lon": np.arange(1,5)},
                    attrs  = {
                        '_FillValue': -999.9,
                        'units'     : ''
                        }
                    )})

fake_lowres.to_netcdf("test_data/lowres_ts_ann_1980-1999.nc")
values = np.ones((5,4)) * 100
#values[1:3,1:3] = 0
fake_sftlf = xr.Dataset({'sftlf': xr.DataArray(
                    data   = values,   # enter data here
                    dims   = ["lat","lon"],
                    coords = {"lat": np.arange(0.5,5.5), "lon": np.arange(1,5)},
                    attrs  = {
                        '_FillValue': -999.9,
                        'units'     : ''
                        }
                    )})
fake_sftlf.to_netcdf("test_data/fake_sftlf_5x4.nc")

In [ ]:
fake_sftlf.sftlf.plot()

In [6]:
multiplier = 0.01
varname = "TS"
sftlf=fake_sftlf
year1=1980
year2=1999
n_realizations=1
RP = 20
T=20
var = fake_lowres
covariate=fake_co2


In [24]:
# python make_rv_climextremes.py varname sftlf_filename return_period covariate covariate_file multiplier year1 year2 root_outfile_name filenames
import cdms2, sys, MV2, locale, numpy, climextremes, cdutil, cdtime, string
print ("python "+ " ".join(sys.argv))

args = argparse.parse_args
filenames = args.modpath
root_outfile = args.output
varname = args.varname
sftlf_file = args.sftlf
sftlf = xr.open_dataset(sftlf_file)
year1 = args.start_year
year2 = args.end_year
RP = args.rp
T = local.atof(RP)
multiplier = locale.atof(args.multiplier)
n_realizations = len(filenames)

#filenames=sys.argv[10:]
#root_outfile_name=sys.argv[9]
#varname=sys.argv[1]
#sftlf_file=cdms2.open(sys.argv[2])
#sftlf=sftlf_file('sftlf')

#year1=sys.argv[7]
#year2=sys.argv[8]


#RP=sys.argv[3]
#T=locale.atof(RP)
# multiplier
#multiplier=locale.atof(sys.argv[6])

#n_realizations=len(filenames)
#f=cdms2.open(filenames[0])
#tim=f['time', time=(year1+'-01-01',year2+'-12-31')]




python /home/ordonez4/miniconda3/envs/pcmdi_metrics_dev/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/ordonez4/.local/share/jupyter/runtime/kernel-1a9031ac-0cfe-41d1-9888-9acf70c4759b.json


NameError: name 'argparse' is not defined

In [ ]:
# I don't know if we need the below stuff anymore:

#u=f.getdimensionunits('time')
#x=f(varname, time=(year1+'-01-01',year2+'-12-31'))
#tim=x.getAxis(0)
#latitude=f.dimensionarray('latitude')
#longitude=f.dimensionarray('longitude')
#latitude=latitude.astype(MV2.float64)
#longitude=longitude.astype(MV2.float64)
#nt=len(tim)
#nlat=latitude.shape[0]
#nlon=longitude.shape[0]
#f.close()

#r = cdtime.reltime(0,tim.units)
#xxyear1=int(1.+r.torel('years since 1').value+tim[0])
#xxyear2=int(1.+r.torel('years since 1').value+tim[nt-1])
#xyear1=string.zfill(xxyear1,4)
#xyear2=string.zfill(xxyear2,4)

covariate_names = args.covariate_names
covariate_file = args.covariate_file
covariate = xr.open_dataset(covariate_file)
if covariate_name == "co2mass":
    l_cov = len(covariate.time) # TODO: probably need to add a variable call here
# Add check that covariate length matches test variable length in time

#covariate_name=sys.argv[4]
#covariate_file=cdms2.open(sys.argv[5])
#covariate=covariate_file(covariate_name, time=(xyear1+'-01-01',xyear2+'-12-31'))
#if covariate_name=='co2mass': covariate=cdutil.YEAR(covariate)
#l_cov=len(covariate)

#if nt!=len(covariate):
#    print ("Error: covariate length",len(covariate)," not equal to variable time dimension length",nt)
#    exit()

## Not sure if I'll need these allocations yet:


# Allocate temporary arrays
#nsample=nt*n_realizations
#var=MV2.zeros((nsample,nlat,nlon),MV2.float)
#replicateIndex=MV2.ones(nsample,MV2.int)
#covariates=MV2.zeros(nsample,MV2.float)

# Allocate output arrays
#return_value=1.e20*MV2.ones((nt,nlat,nlon),MV2.float)
#standard_error=1.e20*MV2.ones((nt,nlat,nlon),MV2.float)


In [146]:
# var: Concatenate all ensemble members (stack in first/time dimension), then
#      multiply by multiplier. Might need to do something with time index of stacked vars
# replicateIndex: realization number, saved in matrix form to track concatenated var
# covariates: covariate time series, stacked in time to match ensemble member stack
#file_list = [xr.open_dataset(filenames[n]) for n in range(0,n_realizations)] # chunks option?
#var = xr.concat(file_list,dim="time")
var = multiplier * var
covariate_list = [covariate for n in range(0,n_realizations)]
covariates = xr.concat(covariate_list,dim="time")
replicate_list =[xr.zeros_like(covariates,dtype="int")+n for n in range(1,n_realizations+1)]
replicateIndex = xr.concat(replicate_list,dim="time")

In [111]:
print(var.where(var.lat > -60).where(sftlf.sftlf > 50).TS.to_numpy().shape)
print(covariates.co2.to_numpy().shape)
print(replicateIndex.co2.to_numpy().shape)
print(covariate.co2.to_numpy().shape)

(20, 5, 4)
(20,)
(20,)
(20,)


In [112]:
tmp = climextremes.fit_gev(
            var.where(var.lat > -60).where(sftlf.sftlf > 50).TS.isel(lat=0,lon=0).to_numpy(),
            covariates.co2.to_numpy(),
            returnPeriod   = T,
            nReplicates    = n_realizations,
            replicateIndex = replicateIndex.co2.to_numpy(),
            locationFun    = 1,
            xNew           = covariate.co2.to_numpy()
)

In [113]:
tmp

{'returnValue': array([0.81297971, 0.81551005, 0.81804038, 0.82057072, 0.82310106,
        0.82563139, 0.82816173, 0.83069206, 0.8332224 , 0.83575273,
        0.83828307, 0.8408134 , 0.84334374, 0.84587407, 0.84840441,
        0.85093474, 0.85346508, 0.85599541, 0.85852575, 0.86105608]),
 'se_returnValue': array([0.03470912, 0.03401273, 0.03337871, 0.03281065, 0.03231205,
        0.03188615, 0.03153592, 0.03126388, 0.03107209, 0.03096205,
        0.03093463, 0.03099004, 0.03112784, 0.03134696, 0.03164569,
        0.03202181, 0.03247263, 0.03299509, 0.03358584, 0.03424136]),
 'info': {'convergence': array([0], dtype=int32),
  'counts_names': array(['function', 'gradient'], dtype='<U8'),
  'counts': array([179.,  nan]),
  'message': None,
  'failure': array([0], dtype=int32)}}

In [156]:
%%time
def apply_gev(x,covariates,T,n_realizations,replicateIndex,covariate,multiplier):
    
    try:
        tmp = climextremes.fit_gev(
            x,
            covariates,
            returnPeriod   = 20,
            nReplicates    = n_realizations,
            replicateIndex = replicateIndex,
            locationFun    = 1,
            xNew           = covariate)
    except:
        return (np.zeros(x.shape)*np.nan,np.zeros(x.shape)*np.nan)
    return_value = tmp['returnValue']/multiplier
    # older results may need to be divided by multiplier (it was an omission bug)
    standard_error = tmp['se_returnValue']/multiplier
    return (return_value,standard_error)
        
vdims=list(dict(var.dims).keys())    
return_value = np.zeros(var.TS.shape)
standard_error = np.zeros(var.TS.shape)

for i in range(0,5):
    for j in range(0,4):
        return_value_tmp, standard_error_tmp = xr.apply_ufunc(
            apply_gev,
            var.where(var.lat > -60).where(sftlf.sftlf > 50).TS.isel(lon=j,lat=i),
            covariates.co2,
            T,
            n_realizations,
            replicateIndex.co2,
            covariate.co2,
            multiplier,
            vectorize=False,
            input_core_dims = [["time"],["time"],[],[],["time"],["time"],[]],
            output_core_dims = [["time"],["time"]], # list dimensions for return_value and standard_error
            join = "outer"
        )
        return_value[:,i,j] = return_value_tmp
        standard_error[:,i,j] = standard_error_tmp

#return value and standard_error are NaN where no data (eg ocean)
#return_value is 0 where variable is 0 in time
#return_value is only calculated where sftlf > 50 (land)
#return_value is fit_gev/multiplier when not 0
#also save standard_error



CPU times: user 2.03 s, sys: 1.74 s, total: 3.77 s
Wall time: 977 ms


In [157]:
return_value

array([[[ 82.92930426,  82.96205359,  81.30110305,  83.41867307],
        [ 82.87110763,  83.58795036,  83.46991764, 111.01179374],
        [ 91.21005246,  83.81646748,  83.31341609,  81.62488783],
        [ 87.19356133,  80.13964488,  82.79959398,  81.58018214],
        [ 84.0657228 ,  79.33583454,  83.02462119,  82.61724111]],

       [[ 83.16278239,  83.15650409,  81.63755531,  83.54880589],
        [ 83.1224086 ,  83.74615855,  83.46241429, 111.09144696],
        [ 91.04435013,  83.88296669,  83.45879256,  81.82206157],
        [ 87.24519981,  80.43972768,  83.02486363,  81.86843094],
        [ 84.20017371,  79.59331176,  83.18535181,  82.82625172]],

       [[ 83.39626051,  83.3509546 ,  81.97400757,  83.67893871],
        [ 83.37370957,  83.90436674,  83.45491094, 111.17110017],
        [ 90.87864779,  83.9494659 ,  83.60416903,  82.01923531],
        [ 87.2968383 ,  80.73981048,  83.25013328,  82.15667973],
        [ 84.33462463,  79.85078897,  83.34608242,  83.03526234]],

    

In [107]:
help(climextremes.fit_gev) 

Help on function fit_gev:

fit_gev(y=None, x=None, locationFun=None, scaleFun=None, shapeFun=None, nReplicates=1.0, replicateIndex=None, weights=None, returnPeriod=None, returnValue=None, getParams=False, getFit=False, xNew=None, xContrast=None, maxes=True, scaling=1.0, bootSE=False, bootControl=None, optimArgs=None, optimControl=None, missingFlag=None, initial=None, logScale=None, _normalizeX=True, _getInputs=False, _allowNoInt=True)
    Fit a generalized extreme value model to block maxima or minima 
    
    **description**
    
     Fit a generalized extreme value model, designed specifically for climate data. It includes options for variable weights (useful for local likelihood), as well as for bootstrapping to estimate uncertainties. Results can be returned in terms of parameter values, return values, return periods, return probabilities, and differences in either return values or log return probabilities (i.e., log risk ratios).
    
    **arguments**
    
    y: a numpy array o

In [96]:
# Write results to file

#Output return values
return_value.id=varname+'_return_value'
return_value.units=x.units
return_value.setAxisList(x.getAxisList())

standard_error.id=varname+'rv_standard_error'
standard_error.units=x.units
standard_error.setAxisList(x.getAxisList())

fout=cdms2.open(varname+'_'+RP+'_return_value_climextremes_'+root_outfile_name+'.nc','w')
fout.execute_line="python "+ " ".join(sys.argv)
fout.write(return_value)
fout.write(standard_error)
fout.return_period=RP
fout.sample_size=nsample
fout.nReplicates=n_realizations
fout.covariate=covariate_name
covariate.id=covariate_name
covariate.setAxis(0,x.getAxis(0))
fout.write(covariate)
fout.close()

AttributeError: cannot set attribute 'id' on a 'DataArray' object. Use __setitem__ styleassignment (e.g., `ds['name'] = ...`) instead of assigning variables.

Experimenting with apply_ufunc

In [100]:
def fake_gev(x,y):
    return (x + y)

array1 = xr.DataArray([[0,1,2,4,5],[0,1,2,4,5]], dims=["x","y"],coords={"x":[0.1, 0.2],
                                                       "y": [0.1, 0.2, 0.3, 0.4, 0.5]})
array2 = xr.DataArray([2,0,0,0,0], dims=["y"],coords={"y": [0.1, 0.2, 0.3,0.4,0.5]}) 
xr.apply_ufunc(fake_gev,array1,array2,vectorize=False,input_core_dims=[["x","y"], ["y"]],output_core_dims=[["x","y"]],join="outer")

<xarray.DataArray (x: 2, y: 5)>
array([[2, 1, 2, 4, 5],
       [2, 1, 2, 4, 5]])
Coordinates:
  * x        (x) float64 0.1 0.2
  * y        (y) float64 0.1 0.2 0.3 0.4 0.5

In [10]:
array1.where(array1 > 1)

<xarray.DataArray (x: 3)>
array([nan, nan,  2.])
Coordinates:
  * x        (x) float64 0.1 0.2 0.3

In [ ]:
# Parser
# 1. varname 
# 2. sftlf_filename 
# 3. return_period 
# 4. covariate 
# 5. covariate_file 
# 6. multiplier 
# 7. year1 
# 8. year2 
# 9. root_outfile_name 
# 10. filenames

parser.add_argument(
        "-v",
        "--vars",
        type=str,
        nargs="+",
        dest="vars",
        help="Variables to use",
        required=True,
    )
parser.add_argument(
        "--return_period",
        type=int,
        dest=return_period,
        help="Return period in years",
        required=True)
parser.add_argument(
        "--covariate",
        type=str,
        dest=covariate,
        help="Covariate variable name",
        required=True)
parser.add_argument(
        "--covariate_file",
        type=str,
        dest=covariate_file,
        help="Covariate file name",
        required=True)
parser.add_argument(
        "--multiplier",
        type=float,
        dest=multiplier,
        help="Multipler value",
        required=True)
parser.add_argument(
        "--years",
        type=list,
        dest=year_range,
        help="[start year, end year]",
        required=True)
parser.add_argument(
        "--filename_template",
        dest="filename_template",
        help="Template for climatology files",
        required=False,
    )
parser.add_argument(
        "--sftlf_filename_template",
        dest="sftlf_filename_template",
        help='Filename template for landsea masks ("sftlf")',
        required=False,
    )
parser.add_argument(
        "--output_filename_template",
        dest="sftlf_filename_template",
        help='Filename template for output file',
        required=True,
    )

In [5]:
import numpy as np
from scipy.stats import genextreme

In [7]:
# https://stackoverflow.com/questions/71202562/calculating-return-value-using-generalised-extreme-value-distribution-gev
shape, loc, scale = genextreme.fit(var.where(var.lat > -60).where(sftlf.sftlf > 50).TS.isel(lon=0,lat=0))
# Compute the return levels for several return periods.
return_periods = np.array([5, 10, 20, 50, 100])
return_levels = genextreme.isf(1/return_periods, shape, loc, scale)

In [8]:
return_levels

array([76.34374244, 79.70977206, 83.51750962, 89.45444364, 94.79221802])

In [10]:
shape, loc, scale

(-0.22448669196007415, 71.09903149772221, 2.9409042980546163)